In [ ]:
# Define a function to check if a variable occurs in a term
def occurs_check(var, term):
    if isinstance(term, str):  # if term is a constant
        return False
    if isinstance(term, tuple):  # if term is a function
        for sub_term in term[1:]:
            if occurs_check(var, sub_term):
                return True
    return term == var

# Unification algorithm
def unify(term1, term2, substitution={}):
    if term1 == term2:
        return substitution

    # If either term is a variable
    elif isinstance(term1, str):
        if term1 in substitution:
            return unify(substitution[term1], term2, substitution)
        if occurs_check(term1, term2):
            return None  # Failure due to occurrence check
        substitution[term1] = term2
        return substitution

    elif isinstance(term2, str):
        return unify(term2, term1, substitution)

    # If both terms are functions
    elif isinstance(term1, tuple) and isinstance(term2, tuple):
        # Check if function symbols are the same
        if term1[0] != term2[0]:
            return None  # Failure if predicates differ
        # Recursively unify their arguments
        if len(term1) != len(term2):
            return None  # Failure if arity is different
        for t1, t2 in zip(term1[1:], term2[1:]):
            substitution = unify(t1, t2, substitution)
            if substitution is None:
                return None
        return substitution

    return None

# Test cases
def test_unification():
    # Example 1: Unification of King(x) and King(John)
    print("Unifying King(x) and King(John):")
    result = unify(('King', 'x'), ('King', 'John'))
    print(result)  # Expected: {'x': 'John'}

    # Example 2: Unification of p(f(a), g(Y)) and p(X, X)
    print("\nUnifying p(f(a), g(Y)) and p(X, X):")
    result = unify(('p', 'f', 'a', 'g', 'Y'), ('p', 'X', 'X'))
    print(result)  # Expected: None (Unification fails)

    # Example 3: Unification of p(b, X, f(g(Z))) and p(Z, f(Y), f(Y))
    print("\nUnifying p(b, X, f(g(Z))) and p(Z, f(Y), f(Y)):")
    result = unify(('p', 'b', 'X', ('f', ('g', 'Z'))), ('p', 'Z', ('f', 'Y'), ('f', 'Y')))
    print(result)  # Expected: {'X': 'f(Y)', 'Z': 'b', 'Y': 'g(b)'}

    # Example 4: Unification of prime(11) and prime(y)
    print("\nUnifying prime(11) and prime(y):")
    result = unify(('prime', '11'), ('prime', 'y'))
    print(result)  # Expected: {'y': '11'}

# Run the test cases
test_unification()


Unifying King(x) and King(John):
{'x': 'John'}

Unifying p(f(a), g(Y)) and p(X, X):
None

Unifying p(b, X, f(g(Z))) and p(Z, f(Y), f(Y)):
{'x': 'John', 'b': 'Z', 'X': ('f', 'Y'), 'Y': ('g', 'Z')}

Unifying prime(11) and prime(y):
{'x': 'John', 'b': 'Z', 'X': ('f', 'Y'), 'Y': ('g', 'Z'), '11': 'y'}


In [ ]:
def unify(expr1, expr2, subst={}):
    """
    Unification algorithm to unify two First-Order Logic expressions.

    Parameters:
    - expr1: First expression (list or tuple form)
    - expr2: Second expression (list or tuple form)
    - subst: Substitution dictionary

    Returns:
    - A dictionary representing the substitutions if unification is possible.
    - None if unification fails.
    """
    if subst is None:
        return None
    elif expr1 == expr2:
        return subst
    elif isinstance(expr1, str) and expr1.islower():  # Variable
        return unify_var(expr1, expr2, subst)
    elif isinstance(expr2, str) and expr2.islower():  # Variable
        return unify_var(expr2, expr1, subst)
    elif isinstance(expr1, list) and isinstance(expr2, list) and len(expr1) == len(expr2):
        return unify(expr1[1:], expr2[1:], unify(expr1[0], expr2[0], subst))
    else:
        return None

def unify_var(var, x, subst):
    """
    Unify a variable with a value or another variable.

    Parameters:
    - var: The variable to unify
    - x: The value or variable to unify with
    - subst: Substitution dictionary

    Returns:
    - Updated substitution dictionary
    """
    if var in subst:
        return unify(subst[var], x, subst)
    elif x in subst:
        return unify(var, subst[x], subst)
    elif occurs_check(var, x):
        return None
    else:
        subst[var] = x
        return subst

def occurs_check(var, x):
    """
    Occurs check to prevent infinite loops during unification.

    Parameters:
    - var: Variable being checked
    - x: Expression to check against

    Returns:
    - True if the variable occurs in the expression
    - False otherwise
    """
    if var == x:
        return True
    elif isinstance(x, list):
        return any(occurs_check(var, sub) for sub in x)
    return False

# Example Usage:
expr1 = ["Knows", "John", "x"]
expr2 = ["Knows", "y", "Alice"]

substitution = unify(expr1, expr2)
if substitution:
    print("Unification Successful! Substitution:", substitution)
else:
    print("Unification Failed!")


Unification Successful! Substitution: {'y': 'John', 'x': 'Alice'}
